<a href="https://colab.research.google.com/github/hpt09/ResumeParser/blob/master/ExpExtractionSnorkel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snorkel
!pip install nltk
!pip install spacy
!pip install phonenumbers

     |████████████████████████████████| 153kB 4.6MB/s 
     |████████████████████████████████| 6.7MB 8.6MB/s 
     |████████████████████████████████| 1.8MB 22.4MB/s 
     |████████████████████████████████| 3.8MB 45.2MB/s 
  Created wheel for networkx: filename=networkx-2.3-py2.py3-none-any.whl size=1556409 sha256=4b5c2cf1791bf836a7d6c78a38c946999634688a156c87d6383096f86dd19e5a
  Stored in directory: /root/.cache/pip/wheels/de/63/64/3699be2a9d0ccdb37c7f16329acf3863fd76eda58c39c737af
Successfully built networkx
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: networkx 2.5
    Uninstalling networkx-2.5:
      Succes

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from snorkel.labeling import labeling_function
import nltk
import spacy
import phonenumbers

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Information Extraction

In [ ]:
import pandas as pd
from snorkel.labeling import labeling_function
import nltk

import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
df_spacy = pd.read_csv(r"cleaned_sentences.csv")

In [ ]:
ABSTAIN = -1
NAME = 1
SKILL = 2
EMAIL = 3
PHONE = 4
EXP = 5

In [ ]:
import re
@labeling_function()
def lf_phone_extraction(x):
    x = x.to_string()
    text = x.lower()
    pattern = r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]'    
    match = re.findall(pattern, text)
    if(len(match)> 0):
        print("Found Phone: ", match, " In: ",text)
        return PHONE
    else:
        return ABSTAIN

In [ ]:
import json
# exps = ["experience", "year", "month", "1" , "2" , "3", "4", "5", "6", "7", "8", "9"]
exps = ["year", "month", "day"]
experience_array = []

@labeling_function()
def lf_skill_extraction(x):
    x = x.to_string()
    text = x.lower()
    text = text.strip()
    obj = None
    with open('cleaned_skills.json', 'r') as skills:
        data = skills.read()
        obj = json.loads(data)
    for skill in obj:
        skill = skill.lower()
 #       skill = (" " + skill + " ")
        for word in text.split():
          if(skill == word):
              print("Found Skill: ", skill, " In: ",text)
              for exp in exps:
                if exp in text:
                  experience_array.append(text)
                  return EXP
              return SKILL
    return ABSTAIN

In [ ]:
import re
@labeling_function()
def lf_email_extraction(x):
    x = x.to_string()
    ners = nlp(x)
    text = x.lower()
    pattern = '[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}'
    match = re.findall(pattern, text)
    if(len(match)> 0):
        print("Found Email: ", match, " In: ",text)
        return EMAIL
    else:
        return ABSTAIN

In [ ]:
# TODO: Work on getting less false positives
from spacy.matcher import Matcher
@labeling_function()
def lf_name_extraction(x):
    x = str(x)
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)
    nlp_text = nlp(x)
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    matcher.add('NAME', None, pattern)
    matches = matcher(nlp_text)
    if len(matches)>0:
        for match_id, start, end in matches:
            span = nlp_text[start:end]
            print("Found Name: ", span)
        return NAME
           
    else:
        return ABSTAIN
@labeling_function()
def lf_temp1(x):
    return ABSTAIN
    
@labeling_function()
def lf_temp2(x):
    return ABSTAIN

In [ ]:
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel

In [ ]:
lfs = [lf_skill_extraction, lf_temp1, lf_temp2]

In [ ]:
applier = PandasLFApplier(lfs)

In [ ]:
L_train = applier.apply(df_spacy)


  0%|          | 163/69499 [00:00<01:25, 806.38it/s]

Found Skill:  database  In:  content     database (less than 1 year)
Found Skill:  html  In:  content     html (less than 1 year)
Found Skill:  access  In:  content     access (less than 1 year)
Found Skill:  c++  In:  content     c++
Found Skill:  java  In:  content     java
Found Skill:  .net  In:  content     .net
Found Skill:  html  In:  content     web designing: html
Found Skill:  access  In:  content     database: ms access
Found Skill:  sql  In:  content     ms sql server 2008
Found Skill:  sql  In:  content     having 3.5 years of it experience in sql data...
Found Skill:  sql  In:  content     development   support of ms sql servers in pr...
Found Skill:  database  In:  content     working experience with relational database s...
Found Skill:  sql  In:  content     maintenance and administration of sql server..
Found Skill:  sql  In:  content     experience in upgrading sql server..     good...
Found Skill:  database  In:  content     high availability of database servers usi


  0%|          | 326/69499 [00:00<01:25, 810.03it/s]

Found Skill:  sql  In:  content     installing sql server clients
Found Skill:  sql  In:  content     performed installation of sql server service ...
Found Skill:  sql  In:  content     upgraded databases from sql server 2000 to sq...
Found Skill:  database  In:  content     the database server performance..     used sq...
Found Skill:  sql  In:  content     installing sql server clients
Found Skill:  database  In:  content     database (3 years)
Found Skill:  sql  In:  content     sql (3 years)
Found Skill:  sql  In:  content     sql dba.
Found Skill:  database  In:  content     database tool: sql management studio (mssql)
Found Skill:  database  In:  content     database language: sql
Found Skill:  c++  In:  content     c++
Found Skill:  pl/sql  In:  content     pl/sql
Found Skill:  unity  In:  content     unity
Found Skill:  spark  In:  content     apache spark and scala.. iaas implementation:...
Found Skill:  natural  In:  content     identifying spam messages using natural langu.


  1%|          | 612/69499 [00:00<01:16, 895.55it/s]

...

Found Skill:  c++  In:  content     c++
Found Skill:  .net  In:  content     .net 2.0
Found Skill:  sql  In:  content     sql
Found Skill:  pl/sql  In:  content     pl/sql
Found Skill:  groovy  In:  content     groovy
Found Skill:  java  In:  content     drools and java 1.4.x.
Found Skill:  sql  In:  content     ms sql server 2000
Found Skill:  sql  In:  content     ms sql server 2005.
Found Skill:  xml  In:  content     xml xml
Found Skill:  xpath  In:  content     xpath
Found Skill:  html  In:  content     technology html
Found Skill:  vbscript  In:  content     vbscript
Found Skill:  javascript  In:  content     javascript
Found Skill:  html  In:  content     html
Found Skill:  sql  In:  content     sql dba
Found Skill:  php  In:  content     php
Found Skill:  self  In:  content     satisfaction and self development and help me...



100%|█████████▉| 69263/69499 [01:13<00:00, 980.55it/s]

Found Skill:  scheme  In:  content     logistics and scheme and marketing activities...
Found Skill:  excel  In:  content     enthusiasm and a desire to excel



100%|██████████| 69499/69499 [01:13<00:00, 942.84it/s]

Found Skill:  clean  In:  content     regulate the process during batch and clean i...
Found Skill:  processing  In:  content     upstream processing of h1n1 monovalent vaccin...


In [ ]:
label_model = LabelModel(cardinality=6, verbose=True)

In [ ]:
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)

In [ ]:
df_spacy["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")

In [ ]:
df_spacy["label"]

0       -1
1       -1
2       -1
3       -1
4       -1
        ..
69494   -1
69495   -1
69496   -1
69497   -1
69498   -1
Name: label, Length: 69499, dtype: int64

In [ ]:
df_spacy.to_csv(path_or_buf = 'snorkel_labeled.csv',index=False)
df_spacy = df_spacy[df_spacy.label != ABSTAIN]

In [ ]:
exp_array = []
for e in range(len(experience_array)):
  exp_array.append(experience_array[e][12:])
  #print(experience_array[e])



database (less than 1 year)
html (less than 1 year)
access (less than 1 year)
having 3.5 years of it experience in sql data...
having 3.5 years of it experience in sql data...
database (3 years)
sql (3 years)
java (1 year)
c++ (less than 1 year)
css (less than 1 year).
creo (less than 1 year)
python (less than 1 year)
database (less than 1 year)
java (less than 1 year)
jira (less than 1. year)
css (less than 1 year)
excel (less than 1 year)
ms excel (less than 1 year)
java (3 years)
junit (2 years)
access (less than 1 year)
css (less than 1 year)
database (less than 1 year).
oracle 5 years 2 month     core java developer.
python (2 years)
java (1 year)
css (1 year)
html (1 year)
mysql (1 year)
javascript (less than 1 year)
math and c++ for 4 years.. .
database (3 years)
python (3 years)
sql (3 years)
xml (3 years).
sap abap (2 years).
tcl (4 years)
python (2 years).
abap (2 years)
c++ (less than 1 year)
xml (3 years).
java (3 years)
mysql (3 years)
python (6 years).
chef (1 year)
datab

In [ ]:
import xlwt 
from xlwt import Workbook 
  
# Workbook is created 
wb = Workbook() 
  
# add_sheet is used to create sheet. 
sheet1 = wb.add_sheet('Sheet 1') 

sheet1.write(0, 0, 'EXPERIENCE')

counter=1

for e in exp_array:
  sheet1.write(0, counter,e)
  print(e)
  counter+=1

wb.save('Experience.xls') 

database (less than 1 year)
html (less than 1 year)
access (less than 1 year)
having 3.5 years of it experience in sql data...
having 3.5 years of it experience in sql data...
database (3 years)
sql (3 years)
java (1 year)
c++ (less than 1 year)
css (less than 1 year).
creo (less than 1 year)
python (less than 1 year)
database (less than 1 year)
java (less than 1 year)
jira (less than 1. year)
css (less than 1 year)
excel (less than 1 year)
ms excel (less than 1 year)
java (3 years)
junit (2 years)
access (less than 1 year)
css (less than 1 year)
database (less than 1 year).
oracle 5 years 2 month     core java developer.
python (2 years)
java (1 year)
css (1 year)
html (1 year)
mysql (1 year)
javascript (less than 1 year)
math and c++ for 4 years.. .
database (3 years)
python (3 years)
sql (3 years)
xml (3 years).
sap abap (2 years).
tcl (4 years)
python (2 years).
abap (2 years)
c++ (less than 1 year)
xml (3 years).
java (3 years)
mysql (3 years)
python (6 years).
chef (1 year)
datab

ValueError: ignored

In [ ]:
import csv

with open('Experience.csv','w') as file:
  file.write('Experience')
  file.write('\n\n')

  for e in exp_array:
      file.write(e)
      file.write('\n')